In [1]:
import chainlit as cl
from llama_index.core import (VectorStoreIndex, SimpleDirectoryReader, SimpleKeywordTableIndex, Settings,
                              StorageContext, QueryBundle, get_response_synthesizer)
from llama_index.core.retrievers import BaseRetriever, KeywordTableSimpleRetriever, VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI, AsyncOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from dotenv import load_dotenv

import os, json

In [2]:
load_dotenv()
OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
with open('../Data/final_data.json', 'r') as file:
  data = json.load(file)

In [7]:
for i, val in enumerate(data):
  temp_data = [val]
  with open(f'../Data1/final_data_{i}.json', 'w') as file:
    json.dump(temp_data, file)

In [4]:
loader = SimpleDirectoryReader("../Data1")
documents = loader.load_data()

In [5]:
Settings.chunk_size = 200
Settings.chunk_overlap = 50
Settings.llm = OpenAI(model = "gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
nodes = Settings.node_parser.get_nodes_from_documents(documents=documents)

In [6]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

In [7]:
vector_index = VectorStoreIndex(nodes, storage_context=storage_context)
keyword_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)

2024-05-19 12:44:51 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:44:55 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:44:58 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:45:01 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:45:04 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:45:07 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [8]:
class Custom_Retriever(BaseRetriever):
  def __init__(self, vector_retriever: VectorIndexRetriever,
               keyword_retriever: KeywordTableSimpleRetriever, mode: str = "AND"):
    self.vector_retriever = vector_retriever
    self.keyword_retriever = keyword_retriever
    self.mode = mode
    super().__init__()

  def _retrieve(self, query_bundle: QueryBundle):
    vector_nodes = self.vector_retriever.retrieve(query_bundle)
    keyword_nodes = self.keyword_retriever.retrieve(query_bundle)

    vector_ids = {n.node.node_id for n in vector_nodes}
    keyword_ids = {n.node.node_id for n in keyword_nodes}

    combine_dict = {n.node.node_id: n for n in vector_nodes}
    combine_dict.update({n.node.node_id: n for n in keyword_nodes})

    if self.mode == "AND":
      retrieve_ids = vector_ids.intersection(keyword_ids)
    else:
      retrieve_ids = vector_ids.union(keyword_ids)

    retrieve_nodes = [combine_dict[rid] for rid in retrieve_ids]
    return retrieve_nodes

In [9]:
vector_retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=3)
keyword_retriever = KeywordTableSimpleRetriever(index=keyword_index)
custom_retriever = Custom_Retriever(vector_retriever, keyword_retriever)

response_synthesizer = get_response_synthesizer(streaming=True)

custom_query_engine = RetrieverQueryEngine.from_args(retriever=custom_retriever,
                                           response_synthesizer=response_synthesizer, streaming=True)

In [24]:
response = custom_query_engine.query("How to configure Company in ITSM")

2024-05-16 20:05:39 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-16 20:05:39 - > Starting query: How to configure Company in ITSM
2024-05-16 20:05:39 - query keywords: ['company', 'itsm', 'configure']
2024-05-16 20:05:39 - > Extracted keywords: ['company', 'itsm', 'configure']
2024-05-16 20:05:40 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [11]:
response = custom_query_engine.query("How to configure BPMS Interface in ITSM?")
print(response)

2024-05-19 12:45:42 - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-05-19 12:45:43 - > Starting query: How to configure BPMS Interface in ITSM?
2024-05-19 12:45:43 - query keywords: ['itsm', 'interface', 'bpms', 'configure']
2024-05-19 12:45:43 - > Extracted keywords: ['itsm', 'interface', 'bpms', 'configure']
2024-05-19 12:45:43 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To configure the BPMS Interface in ITSM, you need to enable the ITSM-BPMS interface from the Eric:CustomerInterface form. For the inbound flow, make sure to select the checkbox under Automation categories for Create and Update Change/Problem. This configuration will allow BPMS to create and update Change Requests and Problems for specified customers. For the outbound flow from ITSM to BPMS, ensure the necessary configuration is in place under Eric:CustomerInterface to enable ITSM to send Create CR requests and Update CR requests for specified cust

In [28]:
for text in response.response_gen:
    print(text)